In [34]:
#General Crawler

def SimpleSpider(orders,recipe,output):
    import requests
    import lxml.html
    f = open(orders, 'r')
    dump= f.read()
    f.close()
    l_url = dump.split('\n')
    l_out = []
    for url in l_url:
        if url != '':
            response = requests.get(url)
            html = lxml.html.fromstring(response.content)
            l_res = []
            l_res = html.xpath(recipe)
            l_out.extend(l_res)

    csv_out = "\n".join(l_out)
    f = open(output, 'w')
    f.write(csv_out)
    f.close()

In [27]:
#Execute Link collection crawling

orders = 'target.txt'
recipe = '//a[contains(@href, "entry") and (starts-with(@href,"http://") or starts-with(@href,"https://"))]/@href'
output = 'results.csv'

SimpleSpider(orders,recipe,output)

In [ ]:
# Stations Page
def parseStations(html):
    title = html.xpath('//*[@class ="entry-title"]//a/text()')
    title = title[0][:title[0].find('　')]
    title = title.split('：')

    h3 = html.xpath('//*[@class ="entry-content"]/h3')
    h3_len =len(h3)
    for i in range(h3_len):
        h3_text = h3[i].xpath('text()')
        h5 = h3[i].xpath('following-sibling::h5')
        h5_len =len(h5)
        if h5_len > 1:
            for j in range(h5_len):
                h5_text = h5[j].xpath('text()')
                x = h5[j].xpath('following-sibling::p')
                content = x[0].xpath('descendant::text()')
                a = []
                a.extend(title)
                a.extend(h3_text)
                a.extend(h5_text)
                a.extend(content)
                yield(a)
        else:
            x = h3[i].xpath('following-sibling::p')
            content = x[0].xpath('descendant::text()')
            a = []
            a.extend(title)
            a.extend(h3_text)
            a.extend('-')
            a.extend(content)
            yield(a)

In [ ]:
# Lines Page
def parseLines(html):
    title = html.xpath('//*[@class ="entry-title"]//a/text()')
    title = title[0][:title[0].find('　')]
    title = title.split('：')
    title = title[0]
    content = html.xpath('//*[@class ="entry-content"]//a/text()')
    a = []
    a.append(title)
    a.extend(content)
    yield(a)

In [44]:
import requests
import lxml.html
import pandas as pd

res_lines = []
res_stations = []

f = open('order.txt', 'r')
dump= f.read()
f.close()
l_url = dump.split('\n')
for url in l_url:
    if url != "":
        response = requests.get(url)
        html = lxml.html.fromstring(response.content)
        isStationPage = len(html.xpath('//*[@id ="乗車停車位置案内"]'))
        if isStationPage ==1:
            res_stations.extend(parseStations(html))
        else:
            res_lines.extend(parseLines(html))

df_stations = pd.DataFrame(res_stations)
df_stations = df_stations.drop_duplicates()
df_stations.to_csv('stations.csv', header=None,index=None)

df_lines = pd.DataFrame(res_lines)
df_lines = df_lines.drop_duplicates()
df_lines.to_csv('lines.csv', header=None,index=None)